# Instal·lar SAMBA

In [ ]:
sudo apt update
sudo apt install samba

Parar el daemon de "Samba" \
Un daemon és un procés que s'executa en segon pla. \
\
Comprovem que el procès de Samba (smbd) està executant-se:

In [ ]:
sudo systemctl status smbd

Si està actiu (executant-se), l'aturem

In [ ]:
sudo systemctl stop smbd

# Modificant la configuració de SAMBA

Algunes vegades no farà falta modificar aquest fitxer ja que la configuració per defecte ja ens funcionaria, però 

La ruta (absoluta) fins al fitxer que volem modificar és la següent:

In [ ]:
vi /etc/samba/smb.conf # editor de text VIM
nano /etc/samba/smb.conf # editor de text NANO

Fem una còpia de seguretat (backup) del fitxer de configuració abans de modificar-lo.

Per fer la còpia simplement podeu fer: 



In [ ]:
cp  /etc/samba/smb.conf  /etc/samba/smb.conf.backup
nano /etc/samba/smb.conf

# o per modificar-lo des de zero:

cp  /etc/samba/smb.conf  /etc/samba/smb.conf.backup
rm /etc/samba/smb.conf
nano /etc/samba/smb.conf

# Com utilitzar NANO (bàsic)

Guardar contingut del fitxer:

`CTRL + "O" + ENTER `    

\

Copiar text:

- Selecciona amb `CTRL`

- Retalla amb `CTRL + "K"`

- Enganxa amb `CTRL + "U"` més d'una vegada

\

Desfer els canvis (Undo):

`ALT + "U"` o `ESC + "U"`

\

Desfer els canvis (redo):

`ALT + "E"` o `ESC + "E"`

\

Tancar l'editor NANO:

`CTRL + "X"`

# Sitaxi del fitxer smb.conf

## Configuració global (només en tenim una!) de SAMBA

[**global**] -> `Configuracions que apliquen a tota la configuració de SAMBA`

\

server string = **El meu server de Fitxers** `# Nom del servidor.`

\

workgroup = **DAM1** `# Nom del Grup de treball.`

\

security = **user** `# Cada usuari tindrà la seva pròpia base de dades d’usuaris (noms d’usuari i contrasenyes locals), no tenim cap base centralitzada. Per tant, estem utilitzant un model P2P.`

\

map to guest = **Bad User** `# No fa falta un compte d'usuari per realitzar la connexió. És a dir, completament obert a tothom. Si estàas utilitzant un compte no vàlid/bloquejat/guest et deixaria entrar igual.`

\

name resolve order = **bcast host** `# L'ordre amb el qual comprovarà el nom dels hosts (equips de la xarxa). Primer broadcast. No fa falta entendre aquest concepte de moment.`




In [ ]:
[global]
server string = El meu servidor de fitxers
workgroup = DAM1
security = user
map to guest = Bad User
name resolve order = bcast host


## Configuracions específiques (em podem tenir més d'una) de SAMBA


Carpeta compartida sense restriccions

In [ ]:
[Fitxers oberts]
path = /fitxers_samba/provaoberts
force user = smbusuari
force group = smbgrup

# Definir els permisos en Linux, ja que el format de permisos en Windows no és tan específic!
create mask = 0664 # màscara de permisos que tindràn els fitxers en aquest directori
force create mode = 0664 
directory mask = 0775 # màscara de permisos que tindràn els directoris en aquest directori
force directory mode = 0775
public = yes # accesible per tothom
writable = yes # podem escriure en el directori


In [ ]:
sudo mkdir -p /fitxers_samba/provaoberts   # Paràmetre -p serveix per crear una carpeta i subdirectoris en una mateixa comanda

In [ ]:
ls -l /fitxers_samba

Ara caldria crear el grup "smbgrup" i l'usuari "smbusuari":

In [ ]:
sudo groupadd --system smbgrup   # El paràmetre "--system" significa que serà un grup del sistema en comptes d'un grup normal. L'única diferència és que un grup del sistema té un GID més baix i normalment s'utilitzen per fer manteniment del sistema o automatitzar tasques de còpies de seguretat
sudo useradd --system --group smbgrup smbusuari   # El paràmetre "--system" significa que serà un grup del sistema en comptes d'un grup normal. L'única diferència és que un usuari del sistema té un UID més baix i normalment s'utilitzen per fer manteniment del sistema o automatitzar tasques de còpies de seguretat. 
                                          # Normalment, un usuari del sistema no se li fa login ja que ha de treballar sol en segon plà

La carpeta compartida ha de ser propietat del usuari i grup que hem creat

In [ ]:
sudo chown -R smbusuari:smbgrup /fitxers_samba
sudo chmod -R g+w /fitxers_samba

In [ ]:
sudo systemctl status smbd
sudo systemctl start smbd

sudo systemctl restart smbd # si us heu oblidat de pararlo, en comptes de start cal fer un re-start

Si heu congelat la màquina, us diu que el servei està actiu però no funciona, feu també un **restart**

# SAMBA (SMB) en Windows

## Per a que funcioni SMB en Windows, cal:







**1)** Activar SMB des de "Activar o desactivar caracteristicas de Windows".

**2)** Activar l'ús compartit de fitxers i impresores + activar la detecció de xarxes.

 

1.   Botó dret a "Abrir configuración de red e Internet"
2.   Baixeu a configuración de red avançada, i premeu "Centro de redes y recursos coompartidos".
3.   Ara haurieu de tenir una finestra del panel de control. En les opcions de la part esquerra, premeu "Cambiar configuración de uso compartido avanzado".

I ara ja heu arribat a la finestra que ens interessa. Cal **activar les dues opcions del perfil actual (privat)**.

\

## Una vegada fet el pas anterior, ja us podeu connectar a la carpeta compartida (al servidor Linux)

Podeu accedir a la carpeta compartida creant un accès directe i posan-li l'adreça a la carpeta compartida.


El format de l'adreça a la carpeta compartida és:

**\\IP_SERVIDOR\NOM_CARPETA_COMPARTIDA**

In [ ]:
# Si:

IP_SERVIDOR = 192.168.6.129
NOM_CARPETA_COMPARTIDA = Fitxers oberts

In [ ]:
# Llavors l'adreça seria:

\\192.168.6.129\Fitxers oberts

# Si no sabeu segur el nom de la carpeta, simplement poseu l'adreça al servidor:

\\192.168.6.129\

# SAMBA en Manjaro

Abans d'accedir a SAMBA cal que tingueu instal·lat el client de SAMBA, és a dir, **smbclient**.

Si el teniu, igual que heu fet en Windows, heu de posar l'adreça a la carpeta compartida.

En Manjaro (i també en altres Linux) el format de l'adreça és:

**smb://IP_SERVIDOR/NOM_CARPETA_COMPARTIDA**

In [ ]:
# Si:

IP_SERVIDOR = 192.168.6.129
NOM_CARPETA_COMPARTIDA = Fitxers oberts

In [ ]:
# Llavors l'adreça seria:

smb://192.168.6.129/Fitxers oberts

# Si no sabeu segur el nom de la carpeta, simplement poseu l'adreça al servidor:

smb://192.168.6.129

# PING

Per defecte, VirtualBox dóna la mateixa adreça IP per defecte a tots els dispositius que es connecten per NAT. Això fa que no et puguis comunicar entre màquines virtuals.


Per connectar-vos correctament entre màquines virtuals, cal crear una xarxa NAT separada.

